# Intro to Neural Nets



<p> This notebook demonstrate neural nets and the concept of hidden layers. We also stress the use of <b>Synthetic featires  </b> which are featurs obtained by applying mathematical operations on the input feature set</p>

In [1]:

# Common Import 

import tensorflow as tf
import matplotlib.pyplot as plt # for visualization.
import numpy as np              # Low-level numerical Python library.
import pandas as pd             # Higher-level numerical Python library.
from matplotlib import cm
from matplotlib import gridspec
from numpy import linspace
import sklearn.metrics as metrics
import math


# Exploratory Data Analysis

     * As usual we always have a validation and test set
     * We replace null values with median values or drop them
     * Replacing with median values is more robust to outliers
     * We can also do pairplot using Bokeh or sns to see correlation in data
     
    

In [37]:
# We import data using pandas. Note that pandas library is already in our import list. check cell one
# 
california_data_test = pd.read_csv("housing.csv", nrows=100)

float_cols = [c for c in california_data_test if california_data_test[c].dtype == "float64"]
float32_cols = {c: np.float32 for c in float_cols}

# Read and convert float64  to float32 . This is to prevent warning in tensorflow.
california_data = pd.read_csv("housing.csv", engine='c', dtype=float32_cols)

In [38]:
# Let us view the first few data set
california_data[0:1]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.230003,37.880001,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY


<p>We preprocess the model to add synthetic features</p>

In [43]:
def preprocess_features(california_data):
  """Prepares input features from California housing data set.

  Args:
    california_data: A Pandas DataFrame expected to contain data
      from the California housing data set.
  Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
  """
  selected_features = california_data[
    ["latitude",
     "longitude",
     "housing_median_age",
     "total_rooms",
     "total_bedrooms",
     "population",
     "households",
     "median_income"]]
  processed_features = selected_features.copy()
  # Create a synthetic feature.
  processed_features["rooms_per_person"] = (
    california_data["total_rooms"] /
    california_data["population"])
  return processed_features

def preprocess_targets(california_data):
    """Prepares target features (i.e., labels) from California housing data set.

        Inoput:
        california_housing_dataframe: A Pandas DataFrame expected to contain data
      from the California housing data set.
      Returns:
        A DataFrame that contains the target feature.
    """
    output_targets = pd.DataFrame()
    # Scale the target to be in units of thousands of dollars.
    output_targets["median_house_value"] = (california_data["median_house_value"] / 1000.0)
    return output_targets

In [75]:
training_examples = preprocess_features(california_data.head(12000))
training_examples.describe()

//anaconda/envs/seychelles/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,12000.000000,12000.000000,12000.000000,12000.000000,11883.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,35.317268,-119.264748,30.317083,2486.408447,518.117310,1398.842285,484.634735,3.886227,1.929673
std,1.974712,1.766544,12.038200,2020.250122,398.944611,1073.816772,370.507385,1.965618,1.311660
min,32.669998,-124.349998,1.000000,2.000000,2.000000,3.000000,2.000000,0.499900,0.008576
25%,33.919998,-120.550003,21.000000,1394.000000,NaN,785.000000,274.000000,2.540025,1.419657
50%,34.110001,-118.349998,32.000000,2020.000000,NaN,1150.000000,397.000000,3.516100,1.897663
75%,37.299999,-118.099998,38.000000,2956.000000,NaN,1697.000000,581.000000,4.768300,2.273499
max,41.950001,-114.550003,52.000000,37937.000000,6445.000000,28566.000000,6082.000000,15.000100,55.222221


In [87]:
training_examples =  training_examples.astype(np.float32);
training_examples.dtypes

latitude              float32
longitude             float32
housing_median_age    float32
total_rooms           float32
total_bedrooms        float32
population            float32
households            float32
median_income         float32
rooms_per_person      float32
dtype: object

In [77]:
training_targets = preprocess_targets(california_data.head(12000))
training_targets.describe()

,median_house_value
count,12000.000000
mean,210.213486
std,115.035683
min,14.999000
25%,126.599998
50%,184.250000
75%,265.100006
max,500.001007


In [78]:
training_targets = training_targets.astype(np.float32);
training_targets.dtypes

median_house_value    float32
dtype: object

In [79]:
# Check for Null
training_examples.size

108000

In [80]:
training_examples.isnull().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        117
population              0
households              0
median_income           0
rooms_per_person        0
dtype: int64

In [81]:
# Fill Null in data set
training_examples["total_bedrooms"].fillna(training_examples["total_bedrooms"].median(),inplace = True)

In [21]:
#Check Null is filled to confirm no null in data sets

In [82]:
training_examples.isnull().sum()

latitude              0
longitude             0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
rooms_per_person      0
dtype: int64

In [25]:
# Below is the validation set
# Note that when we filled null before calling describe there was warning.

In [83]:
validation_examples = preprocess_features(california_data.tail(5000))
validation_examples.describe()


//anaconda/envs/seychelles/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,5000.000000,5000.000000,5000.00000,5000.000000,4944.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,37.265697,-121.441063,28.53780,2551.343994,507.263763,1342.506592,478.305786,4.055655,1.999724
std,1.474897,1.143420,13.01032,1785.133545,345.807068,960.676025,324.749268,1.933667,0.663717
min,34.080002,-123.529999,1.00000,8.000000,1.000000,8.000000,1.000000,0.499900,0.002547
25%,37.029999,-122.309998,18.00000,1479.000000,NaN,762.750000,281.000000,2.708100,1.655801
50%,37.520000,-121.889999,27.00000,2194.000000,NaN,1150.000000,409.000000,3.718800,1.985319
75%,37.950001,-120.930000,37.00000,3189.500000,NaN,1663.000000,601.000000,4.987175,2.300078
max,41.950001,-118.370003,52.00000,25187.000000,3521.000000,12427.000000,3589.000000,15.000100,16.838942


In [89]:
validation_examples = validation_examples.astype(np.float32);
validation_examples.dtypes

latitude              float32
longitude             float32
housing_median_age    float32
total_rooms           float32
total_bedrooms        float32
population            float32
households            float32
median_income         float32
rooms_per_person      float32
dtype: object

In [84]:
validation_targets = preprocess_targets(california_data.tail(5000))
validation_targets =  validation_targets.astype(np.float32);

validation_targets.describe()

,median_house_value
count,5000.000000
mean,230.391632
std,124.273170
min,14.999000
25%,128.575005
50%,220.150002
75%,304.550003
max,500.001007


In [90]:
validation_targets =  validation_targets.astype(np.float32);
validation_targets.dtypes

median_house_value    float32
dtype: object

### Building a neural network

General Notes

Use **`hidden_units`** to define the structure of the NN.  The `hidden_units` argument provides a list of ints, where each int corresponds to a hidden layer and indicates the number of nodes in it.  For example, consider the following assignment:

`hidden_units=[3,10]`

The preceding assignment specifies a neural net with two hidden layers:

* The first hidden layer contains 3 nodes.
* The second hidden layer contains 10 nodes.

If we wanted to add more layers, we'd add more ints to the list. For example, `hidden_units=[10,20,30,40]` would create four layers with ten, twenty, thirty, and forty units, respectively.

By default, all hidden layers will use ReLu activation and will be fully connected.

# Try This ? 

## Now its your turn


This example was taken from the work of https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py
Using the training_examples and training_targets as your traininh set and the validation_example and valdation_targets as the validation sets adapt the example pasted below for our model.

Can we adapt this to our data set:
Hints:
        * Use batch function from previous section
        * Import and Use synthetic features 
        * Compare your model
        * The target variable is just one in our case (n_classes=1)
        * Share your opinion/solutions email opealek@gmail.com

In [ ]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))